In [1]:
'''def set_localisation(nb, reg, dpt, com):
    dic_reg = {i: str(val) for i, val in enumerate(reg['REG'])}
    dic_dpt = {i: str(val) for i, val in enumerate(dpt['DEP'])}    
    dic_com = {i: str(val) for i, val in enumerate(com['COM'])}      
    list_random = [random.randint(0,len(com)-1) for i in range(nb)]
    return pd.DataFrame({'region': [com['REG'][i] for i in list_random],
                         'dept': [com['DEP'][i] for i in list_random],
                         'com': [com['COM'][i] for i in list_random]})'''

"def set_localisation(nb, reg, dpt, com):\n    dic_reg = {i: str(val) for i, val in enumerate(reg['REG'])}\n    dic_dpt = {i: str(val) for i, val in enumerate(dpt['DEP'])}    \n    dic_com = {i: str(val) for i, val in enumerate(com['COM'])}      \n    list_random = [random.randint(0,len(com)-1) for i in range(nb)]\n    return pd.DataFrame({'region': [com['REG'][i] for i in list_random],\n                         'dept': [com['DEP'][i] for i in list_random],\n                         'com': [com['COM'][i] for i in list_random]})"

In [2]:
import pandas as pd
import json
import shapely

def get_geom(filename):
    file = open(filename)
    contour = json.load(file)
    file.close()
    geom = []
    for feature in contour['features']:
        code = feature['properties']['code']
        try: 
            poly = shapely.from_geojson(json.dumps(feature['geometry']))
            box = shapely.bounds(poly).tolist()
        except: 
            poly = None
            box = [0.0, 0.0, 0.0, 0.0]
        geom.append((code, box, poly))
    return pd.DataFrame.from_records(geom, columns=['code', 'box', 'poly'])

def in_box(point, box):
    return box[0] <= point[0] <= box[2] and box[1] <= point[1] <= box[3]

def get_com(coord, data_ref):
    reg, dpt, com, reg_geo, dpt_geo, com_geo, comidx = data_ref
    reg_ok = reg_geo.apply(lambda x: in_box(coord, x['box']), axis=1)
    reg_geo_ok = list(reg_geo[reg_ok]['code'])
    
    dpt_sel = list(dpt[dpt['REG'].isin(reg_geo_ok)]['DEP'])
    dpt_sel_geo = dpt_geo[dpt_geo['code'].isin(dpt_sel)]
    dpt_ok = dpt_sel_geo.apply(lambda x: in_box(coord, x['box']), axis=1)
    dpt_geo_ok = list(dpt_sel_geo[dpt_ok]['code'])

    com_sel = list(com[com['DEP'].isin(dpt_geo_ok)]['COM'])
    com_sel_geo = com_geo[com_geo['code'].isin(com_sel)]
    com_ok = com_sel_geo.apply(lambda x: in_box(coord, x['box']), axis=1)
    com_geo_ok = list(com_sel_geo[com_ok]['code'])
    
    for commune in com_geo_ok:
        poly = com_geo.loc[com_geo['code'] == commune, :]['poly'].iloc[0]
        if shapely.contains_xy(poly, coord[0], coord[1]):
            # com_att=comidx.loc[commune]
            # return (commune, com_att['DEP'], com_att['REG'])
            return commune
    # return (None, None, None)
    return None

In [3]:
chemin = "D:/philippe/python ESstandard/IRVE/sources/data_quali/"
columns=['code', 'box', 'poly']
reg_geo = get_geom(chemin + 'regions-version-simplifiee.geojson')
dpt_geo = get_geom(chemin + 'departements-version-simplifiee.geojson')
com_geo = get_geom(chemin + 'communes-version-simplifiee.geojson')
reg = pd.read_csv(chemin + 'v_region_2024.csv')
dpt = pd.read_csv(chemin + 'v_departement_2024.csv').astype('str')
com = pd.read_csv(chemin + 'v_commune_2024.csv', dtype='str')
comidx = com.set_index('COM')
data_ref = (reg, dpt, com, reg_geo, dpt_geo, com_geo, comidx)

In [17]:
dpt['REG'] = dpt['REG'].astype('int')
dpt.loc[dpt['REG'] < 10, :]
dpt[90:110]

,DEP,REG,CHEFLIEU,TNCC,NCC,NCCENR,LIBELLE
90,90,27,90010,2,TERRITOIRE DE BELFORT,Territoire de Belfort,Territoire de Belfort
91,91,11,91228,5,ESSONNE,Essonne,Essonne
92,92,11,92050,4,HAUTS DE SEINE,Hauts-de-Seine,Hauts-de-Seine
93,93,11,93008,3,SEINE SAINT DENIS,Seine-Saint-Denis,Seine-Saint-Denis
94,94,11,94028,2,VAL DE MARNE,Val-de-Marne,Val-de-Marne
95,95,11,95500,2,VAL D OISE,Val-d'Oise,Val-d'Oise
96,971,1,97105,3,GUADELOUPE,Guadeloupe,Guadeloupe
97,972,2,97209,3,MARTINIQUE,Martinique,Martinique
98,973,3,97302,3,GUYANE,Guyane,Guyane
99,974,4,97411,0,LA REUNION,La Réunion,La Réunion


In [4]:
coord = [5, 47]
#com_ok, dep_ok, reg_ok = get_com(coord, data_ref)
com_ok = get_com(coord, data_ref)

#print(com_ok, dep_ok, reg_ok)
print(com_ok)
print(com_geo[com_geo['code']==com_ok]['box'].iloc[0])

21189
[4.972450762618274, 46.999128507562766, 5.053123943966335, 47.02143928952511]


In [5]:
from time import time
loc = pd.read_csv(chemin +'export_localisation_2024-07-25.csv')
t0 = time()
loc['coord'] = loc.apply(lambda x: json.loads(x['coordonneesxy']), axis=1)
print(len(loc), time() - t0)
t0 = time()
loc['commune'] = loc.apply(lambda x: get_com(x['coord'], data_ref), axis=1)
duree = time()-t0
print(len(loc), duree)
loc['commune']

7916 0.05850338935852051
7916 106.23013663291931


0       77018
1       59512
2       74006
3       74010
4       74010
        ...  
7911    31056
7912    81105
7913    40310
7914    13055
7915    92062
Name: commune, Length: 7916, dtype: object

In [6]:
loc['longitude'] = loc.apply(lambda x: x['coord'][0], axis=1)
loc['latitude'] = loc.apply(lambda x: x['coord'][1], axis=1)
loc.to_csv(chemin + 'loc_with_commune.csv')

In [7]:
poly = com_geo.loc[com_geo['code'] == '21193', :]['poly'].iloc[0]
shapely.contains_xy(poly, coord[0], coord[1])

False

In [8]:
comidx = com.set_index('COM')
comidx.loc['01002']['REG']

'84'

In [30]:
filename = chemin + 'regions-avec-outre-mer.geojson'
file = open(filename)
contour = json.load(file)
file.close()
print(type(contour['features']))

obj = contour['features'][0]
print(obj['type'], obj['properties'])

amerique = ['1', '01', '2', '02', '3', '03']
afrique = 
contour_amerique = {'type': 'FeatureCollection', 
                    'features': [feature for feature in contour['features'] if feature['properties']['code'] in amerique]}
contour_amerique2 = {'type': 'FeatureCollection', 
                    'features': [feature for feature in contour['features'] if feature['properties']['code'] in amerique]}
# print(contour_amerique2 == contour_amerique)
with open(chemin + 'regions-amerique.geojson', 'w', encoding ='utf8') as json_file:
    json.dump(contour_amerique, json_file)

<class 'list'>
Feature {'code': '11', 'nom': 'Île-de-France'}
